In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
from IPython.display import clear_output
import json
from statistics import mode

In [2]:
# Array de UMA lista de listas

In [3]:
def MAV_EMG(df, WINDOW_SIZE):
    MAV_values = []
    for j in range(8):
        MAV_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i][:WINDOW_SIZE]['EMG_s'+ str(j)].reset_index(drop=True)
                MAV_temp.append(mean(abs(samples_temp)))
        MAV_values.append(MAV_temp)
    return MAV_values

def WL_EMG(df, WINDOW_SIZE):
    WL_values = []
    for j in range(8):
        WL_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i][:WINDOW_SIZE-1]['EMG_s'+ str(j)].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i][1:WINDOW_SIZE]['EMG_s'+ str(j)].reset_index(drop=True)
                WL_temp.append(sum(abs(samples_temp_plus-samples_temp)))
        WL_values.append(WL_temp)
    return WL_values

def SSC_EMG(df, WINDOW_SIZE, limiar):
    SSC_values = []
    for j in range(8):
        SSC_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][2:WINDOW_SIZE].reset_index(drop=True)
                samples_temp_minus = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-2].reset_index(drop=True)
                SSC_temp.append(sum(((samples_temp - samples_temp_minus)*(samples_temp - samples_temp_plus)>limiar).astype(int)))
        SSC_values.append(SSC_temp)
    return SSC_values

def ZC_EMG(df, WINDOW_SIZE, limiar):
    ZC_values = []
    for j in range(8):
        ZC_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE].reset_index(drop=True)
                op1 = (samples_temp*samples_temp_plus) >= limiar
                op2 = (samples_temp - samples_temp_plus) >= limiar
                ZC_temp.append(sum(op1!=op2))
        ZC_values.append(ZC_temp)
    return ZC_values

#--------------------------------------------------------------------------------------------------------------------------

def IEMG_EMG(df, WINDOW_SIZE):
    IEMG_values = []
    for j in range(8):
        IEMG_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                IEMG_temp.append(sum(abs(samples_temp)))
        IEMG_values.append(IEMG_temp)
    return IEMG_values

def DASDV_EMG(df, WINDOW_SIZE):
    DASDV_values = []
    for j in range(8):
        DASDV_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE].reset_index(drop=True)
                DASDV_temp.append(sum(pow(samples_temp_plus-samples_temp,2))/len(samples_temp))
                if (np.isnan(sum(pow(samples_temp_plus-samples_temp,2))/len(samples_temp))):
                    print(len(samples_temp))
                    print(len(samples_temp_plus))
                    print(len(df[df['signal'] == i]))
                    print()
        DASDV_values.append(DASDV_temp)
    return DASDV_values

def WAMP_EMG(df, WINDOW_SIZE, limiar):
    WAMP_values = []
    for j in range(8):
        WAMP_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) > WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE].reset_index(drop=True)
                WAMP_temp.append(sum(abs(samples_temp-samples_temp_plus)>=limiar))
        WAMP_values.append(WAMP_temp)
    return WAMP_values

def AR4_EMG(df, WINDOW_SIZE):
    AR4_values = []
    for j in range(8):
        AR4_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                rho, sigma = sm.regression.linear_model.burg(samples_temp, order=4)
                AR4_temp.append(rho.tolist())
        AR4_values.append(AR4_temp)
    return AR4_values

def VORDER_EMG(df, WINDOW_SIZE, v):
    VORDER_values = []
    for j in range(8):
        VORDER_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                VORDER_temp.append(abs(pow(sum(pow(samples_temp,v))/len(samples_temp),1/v)))
        VORDER_values.append(VORDER_temp)
    return VORDER_values

#--------------------------------------------------------------------------------------------------------------------------

def VAR_EMG(df, WINDOW_SIZE):
    VAR_values = []
    for j in range(8):
        VAR_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) > WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                VAR_temp.append(sum(pow(samples_temp,2))/(len(samples_temp)-1))
        VAR_values.append(VAR_temp)
    return VAR_values

def RMS_EMG(df, WINDOW_SIZE):
    RMS_values = []
    for j in range(8):
        RMS_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) > WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                RMS_temp.append(pow(sum(pow(samples_temp,2))/len(samples_temp),1/2))
        RMS_values.append(RMS_temp)
    return RMS_values

In [4]:
def GROUP_1 (df, WINDOW_SIZE, limiar, label):
    # MAV, WL, SSC E ZC
    MAV_LIST = MAV_EMG(df, WINDOW_SIZE)
    WL_LIST  = WL_EMG(df, WINDOW_SIZE)
    SSC_LIST = SSC_EMG(df, WINDOW_SIZE,limiar)
    ZC_LIST  = ZC_EMG(df, WINDOW_SIZE,limiar)
    dict_list = []
    for j in range(min([max(df['signal']), 10])):
        if(len(df[df['signal'] == j+1]) >= WINDOW_SIZE):
            entry = [label]
            sensor_list = []
            for i in range(8):
                sensor_list.append(MAV_LIST[i][j])
                sensor_list.append(WL_LIST[i][j])
                sensor_list.append(SSC_LIST[i][j])
                sensor_list.append(ZC_LIST[i][j])
                if(np.isnan(MAV_LIST[i][j])):
                    print('MAV')
                if(np.isnan(WL_LIST[i][j])):
                    print('WL')
                if(np.isnan(SSC_LIST[i][j])):
                    print('SSC')
                if(np.isnan(ZC_LIST[i][j])):
                    print('ZC')
            entry.append(sensor_list)
            dict_list.append(entry)
    df_car = pd.DataFrame(dict_list)
    df_car.columns = ['label', 'feature']
    return df_car

In [5]:
def GROUP_2 (df, WINDOW_SIZE, limiar, v, label):
#    IEMG_LIST   = IEMG_EMG(df, WINDOW_SIZE)
    DASDV_LIST  = DASDV_EMG(df, WINDOW_SIZE)
#    WAMP_LIST   = WAMP_EMG(df, WINDOW_SIZE, limiar)
#    VORDER_LIST = VORDER_EMG(df, WINDOW_SIZE, v)
    AR4_LIST = AR4_EMG(df, WINDOW_SIZE)
    dict_list = []
    for j in range(min([max(df['signal']), 10])):
        if(len(df[df['signal'] == j+1]) >= WINDOW_SIZE):
            entry = [label]
            sensor_list = []
            for i in range(8):
                sensor_list.append(IEMG_LIST[i][j])
                sensor_list.append(DASDV_LIST[i][j])
                sensor_list.append(WAMP_LIST[i][j])
                sensor_list.append(VORDER_LIST[i][j])
                sensor_list.extend(AR4_LIST[i][j])
                if(np.isnan(IEMG_LIST[i][j])):
                    print('IEMG_LIST')
                if(np.isnan(DASDV_LIST[i][j])):
                    print('DASDV_LIST')
                if(np.isnan(WAMP_LIST[i][j])):
                    print('WAMP_LIST')
                if(np.isnan(VORDER_LIST[i][j])):
                    print('VORDER_LIST')
                if(any(np.isnan(AR4_LIST[i][j]))):
                    print('AR4_LIST')
            entry.append(sensor_list)
            dict_list.append(entry)
    df_car = pd.DataFrame(dict_list)
    df_car.columns = ['label', 'feature']
    return df_car

In [6]:
def calculate_general_precision(cm):
    precision = 0
    correct = 0
    total = sum(sum(cm))
    for i in range(len(cm)):
        correct = sum([cm[i][i], correct])
    precision = correct/total
    return precision 

In [7]:
with open('best_parameters.txt') as json_file:
    best_parameters = json.load(json_file)

In [8]:
best_parameters

{'tree_group_number': 1,
 'tree_WINDOW_SIZE': 250,
 'precision_tree': 0.8277310924369747,
 'bag_group_number': 2,
 'bag_WINDOW_SIZE': 250,
 'precision_bag': 0.8109243697478992,
 'rf_group_number': 1,
 'rf_WINDOW_SIZE': 250,
 'precision_rf': 0.907563025210084,
 'svm_group_number': 1,
 'svm_WINDOW_SIZE': 250,
 'precision_svm': 0.9369747899159664,
 'lda_group_number': 2,
 'lda_WINDOW_SIZE': 250,
 'precision_lda': 0.7983193277310925,
 'tree_depth': 10,
 'bag_n_estimators': 110,
 'bag_max_samples': 150,
 'rf_depth': 12,
 'svm_gamma': 0.0001,
 'svm_C': 100,
 'best_group_number': 1,
 'best_WINDOW_SIZE': 250}

In [24]:
def process_activation(sample, window_size, tree_clf, bag_clf, rf_clf, lda_clf, svm_clf, group_number, gesture, subject, precision_dict):
    if len(sample) > 0:
        if group_number == 1:
            df_car_temp = GROUP_1(sample, window_size, 0.001, 'G1')
        if group_number == 2:
            df_car_temp = GROUP_2(sample, window_size, 0.001, 3, 'G1')
        sample_input = df_car_temp['feature'].to_list()
        tree = tree_clf.predict(sample_input)
        bag  = bag_clf.predict(sample_input)
        rf   = rf_clf.predict(sample_input)
        lda  = lda_clf.predict(sample_input)
        svm  = svm_clf.predict(sample_input)
        print('###')
        print(gesture)
        print('###')
        print(tree[0])
        print(bag[0])
        print(rf[0])
        print(lda[0])
        print(svm[0])
        print('---')
        if (tree[0] == gesture):
            precision_dict.update({subject + '_' + gesture + '_' + 'tree_correct'  : precision_dict[subject + '_' + gesture + '_' + 'tree_correct'] + 1})
        if (bag[0] == gesture):
            precision_dict.update({subject + '_' + gesture + '_' + 'bag_correct'  : precision_dict[subject + '_' + gesture + '_' + 'bag_correct'] + 1})
        if (rf[0] == gesture):
            precision_dict.update({subject + '_' + gesture + '_' + 'rf_correct'  : precision_dict[subject + '_' + gesture + '_' + 'rf_correct'] + 1})
        if (lda[0] == gesture):
            precision_dict.update({subject + '_' + gesture + '_' + 'lda_correct'  : precision_dict[subject + '_' + gesture + '_' + 'lda_correct'] + 1})
        if (svm[0] == gesture):
            precision_dict.update({subject + '_' + gesture + '_' + 'svm_correct'  : precision_dict[subject + '_' + gesture + '_' + 'svm_correct'] + 1})
        precision_dict.update({subject + '_' + gesture + '_' + 'total'  : precision_dict[subject + '_' + gesture + '_' + 'total'] + 1})
    
    return

In [34]:
for subject_number in range(5):
    for gesture_number in range(1,6):
        
        precision_dict .update({  's' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'tree_correct': 0,
                                  's' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'bag_correct' : 0,
                                  's' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'rf_correct'  : 0,
                                  's' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'lda_correct' : 0,
                                  's' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'svm_correct' : 0,
                                  's' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'total'       : 0})

calc_mean_start = 50

group_number = best_parameters['best_group_number']
WINDOW_SIZE  = best_parameters['best_WINDOW_SIZE']


for subject_number in range(5):
    df_car = pd.DataFrame(columns=['label', 'feature'])
    for file_test_number in range (10):
        for file_number in range(10):
            for gesture_number in range(1,6):
                if file_number != file_test_number:
    #                 file_segmented  = "C:/Users/User/Desktop/nao_excluir/Matheus_Taborda/Matheus/Subjects_segmented/s_"+str(subject_number)+"/G"+str(gesture_number) +"/Emg_"+str(file_number)+".csv"
                    file_segmented  = "D:/matc/Documents/Projects/Subjects_segmented/s_"+str(subject_number)+"/G"+str(gesture_number) +"/Emg_"+str(file_number)+".csv"

                    df = pd.read_csv(file_segmented)
                    df.drop('Unnamed: 0', axis = 1, inplace = True)
                    label  ='G' + str(gesture_number)
                    # Normalizando valores:
                    for i in range(8):
                        df['EMG_s'+str(i)] = df['EMG_s'+str(i)].apply(lambda x: x/128)
                    if (max(df.signal)>0):
                        if group_number == 1:
                            df_temp = GROUP_1(df, WINDOW_SIZE, 0.001, label)
                        if group_number == 2:
                            df_temp = GROUP_2(df, WINDOW_SIZE, 0.001, 3, label)
                        df_car = df_car.append(df_temp.copy())

        df_train = df_car.reset_index(drop = True)

        X_train = df_train['feature'].to_list()
        y_train = df_train['label'].to_list()

        tree_clf = DecisionTreeClassifier(max_depth=best_parameters['tree_depth'])
        tree_clf.fit(X_train, y_train)

        bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = best_parameters['bag_n_estimators'], max_samples = 20, bootstrap = True, n_jobs=-1)
        bag_clf.fit(X_train, y_train)

        rf_clf = RandomForestClassifier(max_depth=best_parameters['rf_depth'], random_state=42)
        rf_clf.fit(X_train, y_train)

        svm_clf = SVC(kernel="rbf", gamma= best_parameters['svm_gamma'], C=best_parameters['svm_C'])
        svm_clf.fit(X_train, y_train)

        lda_clf = LinearDiscriminantAnalysis()
        lda_clf.fit(X_train, y_train)

        for gesture_number in range(1,6):

        #     file_sampled  = "C:/Users/User/Desktop/nao_excluir/Matheus_Taborda/Matheus/Subjects_sampled/s_"+str(subject_number)+"/G"+str(gesture_number)+"/Emg_"+str(file_number)+".csv"
            # file_sampled  = "C:/Users/Bioteca/Desktop/tcc/Matheus/Subjects_sampled/s_"+str(subject_number)+"/G"+str(gesture_number)+"/Emg_"+str(file_number)+".csv"            
            file_sampled  = "D:/matc/Documents/Projects/Subjects_sampled/s_"+str(subject_number)+"/G"+str(gesture_number)+"/Emg_"+str(file_test_number)+".csv"

            df2 = pd.read_csv(file_sampled)

            for j in df2.drop(['timestamp'],axis=1).columns:
                filtered_signal = df2[j].to_list()
                filtered_signal_2 = filtered_signal.copy()
                for i in range(2,len(df2)):
                    if filtered_signal[i] != 0 :
                        filtered_signal[i] = filtered_signal_2[i-1]**2 -filtered_signal_2[i]*filtered_signal_2[i-2]
                df2[j + '_filtered'] = filtered_signal

            for j in range(0,8):
                df2['EMG_s'+str(j)+'_media_movel'] = df2['EMG_s'+str(j)+'_filtered'].rolling(100).mean()

            df2['total_nrg'] = pd.Series(0, index=np.arange(len(df2)))
            for j in range(0,8):
                df2.total_nrg        = df2.total_nrg        + df2['EMG_s'+str(j)+'_media_movel'].apply(lambda x : abs(x))

            for i in range(8):
                df2['EMG_s'+str(i)] = df2['EMG_s'+str(i)].apply(lambda x: x/128)

            calculated_mean = mean(df2.total_nrg[df2.total_nrg.first_valid_index():].head(calc_mean_start).dropna())

            signal_temp = []

            HISTERESE = 1.6

            THRESHOLD_IN = 1.7

            THRESHOLD_OUT = 2

            sample_counter = 0

            is_signal = False

            activation_start = 0

            activation_end   = 0

            for i in range(calc_mean_start,len(df2)):
                # print(calculated_mean)
                if is_signal:
                    if (df2.total_nrg[i] > THRESHOLD_OUT *calculated_mean):
                        if sample_counter == WINDOW_SIZE:
                            activation_end = i - 1
                            print(activation_end - activation_start)
                            df_sample = df2[activation_start:activation_end].copy()
                            df_sample.reset_index(drop=True)
                            df_sample['signal'] = 1

                            process_activation(df_sample, WINDOW_SIZE, tree_clf, bag_clf, rf_clf, lda_clf, svm_clf, group_number, "G"+str(gesture_number), "s"+str(subject_number), precision_dict)
                    else:
                        is_signal = False
                    sample_counter = sample_counter + 1
                else:
                    if df2.total_nrg[i] > THRESHOLD_IN * calculated_mean:
                        activation_start = i
                        is_signal = True
                    else:
                        if df2.total_nrg[i] < HISTERESE* calculated_mean:
                            calculated_mean = (calculated_mean*(i-1)+df2.total_nrg[i]*5)/(i+4)
                    sample_counter = 0

250
  label                                            feature
0    G1  [0.03590625, 14.609375, 121, 125, 0.06559375, ...
###
G1
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.03290625, 13.234375, 87, 125, 0.05425, 21.4...
###
G1
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.03109375, 12.703125, 92, 115, 0.058875, 22....
###
G1
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.03496875, 14.828125, 121, 118, 0.064375, 25...
###
G1
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.04021875, 16.1953125, 128, 127, 0.0705625, ...
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.03975, 16.5390625, 138, 118, 0.07509375, 29...
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.0405625, 17.0

250
  label                                            feature
0    G1  [0.20915625, 85.6015625, 164, 130, 0.3675, 147...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.12803125, 49.8125, 161, 121, 0.1300625, 49....
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.16015625, 65.5234375, 166, 119, 0.24678125,...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.10046875, 38.1484375, 159, 127, 0.1385625, ...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.1388125, 52.2265625, 157, 119, 0.2040625, 7...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.14559375, 58.6171875, 176, 124, 0.24075, 95...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.1665, 71.3281

  label                                            feature
0    G1  [0.12059375, 47.7265625, 168, 124, 0.16153125,...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.13228125, 55.796875, 176, 129, 0.1710625, 6...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.12090625, 52.0078125, 182, 123, 0.1406875, ...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.115125, 45.3515625, 158, 126, 0.1495, 60.56...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.11090625, 45.3828125, 161, 120, 0.1501875, ...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.1351875, 56.5390625, 172, 131, 0.1535, 63.3...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.12590625, 52.0390

  label                                            feature
0    G1  [0.1113125, 46.5390625, 176, 120, 0.142, 60.24...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.10028125, 40.25, 145, 128, 0.1259375, 51.29...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.07203125, 29.890625, 149, 130, 0.06875, 27....
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.0745, 30.2734375, 145, 127, 0.0876875, 36.0...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.07190625, 28.7578125, 141, 127, 0.0916875, ...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.06803125, 28.015625, 123, 111, 0.096375, 38...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.07903125, 32.2109

250
  label                                            feature
0    G1  [0.06965625, 28.703125, 144, 121, 0.06890625, ...
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.07928125, 33.7734375, 154, 127, 0.090375, 3...
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.04171875, 15.96875, 114, 123, 0.06553125, 2...
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.12034375, 49.7421875, 167, 121, 0.148375, 6...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.09225, 38.0078125, 164, 134, 0.1456875, 60....
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.1071875, 45.078125, 170, 115, 0.13646875, 5...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.0863125, 36.1

...

###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.0465, 18.3984375, 115, 117, 0.04953125, 20....
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.0544375, 22.1171875, 118, 127, 0.04790625, ...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.0554375, 21.6796875, 124, 125, 0.04271875, ...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.036875, 14.0234375, 110, 124, 0.048, 20.906...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.013, 4.8046875, 21, 99, 0.03190625, 13.0546...
###
G4
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.20659375, 84.546875, 172, 126, 0.2868125, 1...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            fea

  label                                            feature
0    G1  [0.22571875, 92.078125, 163, 130, 0.29646875, ...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.1748125, 72.3515625, 177, 134, 0.20328125, ...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.01678125, 7.0703125, 65, 115, 0.06953125, 2...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.027375, 10.703125, 82, 112, 0.08090625, 33....
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.02215625, 8.703125, 83, 109, 0.09728125, 41...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.01978125, 7.7578125, 67, 115, 0.07290625, 3...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.01578125, 6.36718

###
G2
###
G2
G2
G2
G2
G2
---
250
  label                                            feature
0    G1  [0.02175, 8.8046875, 79, 113, 0.04428125, 17.8...
###
G2
###
G2
G2
G2
G2
G2
---
250
  label                                            feature
0    G1  [0.0224375, 8.484375, 71, 112, 0.03521875, 14....
###
G2
###
G2
G2
G2
G2
G2
---
250
  label                                            feature
0    G1  [0.01628125, 6.3125, 46, 119, 0.02934375, 12.1...
###
G2
###
G2
G2
G2
G2
G2
---
250
  label                                            feature
0    G1  [0.0155625, 6.1328125, 54, 105, 0.02675, 11.31...
###
G2
###
G2
G2
G2
G2
G2
---
250
  label                                            feature
0    G1  [0.02259375, 8.796875, 68, 112, 0.02440625, 10...
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            feature
0    G1  [0.01890625, 7.6796875, 65, 115, 0.0211875, 8....
###
G3
###
G3
G3
G3
G3
G3
---
250
  label                                            fea

###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.06409375, 25.40625, 160, 113, 0.0438125, 18...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.07721875, 30.703125, 143, 121, 0.05, 20.812...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.06796875, 27.6796875, 153, 121, 0.0484375, ...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.07321875, 30.8984375, 163, 131, 0.05046875,...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.07553125, 32.34375, 168, 126, 0.05159375, 2...
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            feature
0    G1  [0.0661875, 26.7578125, 159, 127, 0.05675, 23....
###
G4
###
G4
G4
G4
G4
G4
---
250
  label                                            fea

  label                                            feature
0    G1  [0.11946875, 50.328125, 170, 123, 0.15421875, ...
###
G5
###
G5
G5
G5
G5
G5
---
250
  label                                            feature
0    G1  [0.021875, 8.609375, 84, 120, 0.0865625, 35.11...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.01815625, 6.7578125, 52, 118, 0.0718125, 29...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.023875, 9.53125, 96, 112, 0.07103125, 29.14...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.02496875, 9.8125, 87, 118, 0.10103125, 42.6...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.0211875, 8.4296875, 77, 120, 0.066375, 28.3...
###
G1
###
G1
G1
G1
G1
G1
---
250
  label                                            feature
0    G1  [0.0241875, 10.03906

In [35]:
precision_dict

{'s4_G5_tree_correct': 88,
 's4_G5_bag_correct': 87,
 's4_G5_rf_correct': 88,
 's4_G5_lda_correct': 88,
 's4_G5_svm_correct': 88,
 's4_G5_total': 88,
 's0_G1_tree_correct': 63,
 's0_G1_bag_correct': 60,
 's0_G1_rf_correct': 63,
 's0_G1_lda_correct': 63,
 's0_G1_svm_correct': 63,
 's0_G1_total': 67,
 's0_G2_tree_correct': 9,
 's0_G2_bag_correct': 1,
 's0_G2_rf_correct': 11,
 's0_G2_lda_correct': 11,
 's0_G2_svm_correct': 11,
 's0_G2_total': 11,
 's0_G3_tree_correct': 50,
 's0_G3_bag_correct': 46,
 's0_G3_rf_correct': 50,
 's0_G3_lda_correct': 49,
 's0_G3_svm_correct': 50,
 's0_G3_total': 50,
 's0_G4_tree_correct': 80,
 's0_G4_bag_correct': 80,
 's0_G4_rf_correct': 80,
 's0_G4_lda_correct': 80,
 's0_G4_svm_correct': 80,
 's0_G4_total': 80,
 's0_G5_tree_correct': 77,
 's0_G5_bag_correct': 77,
 's0_G5_rf_correct': 78,
 's0_G5_lda_correct': 78,
 's0_G5_svm_correct': 78,
 's0_G5_total': 78,
 's1_G1_tree_correct': 75,
 's1_G1_bag_correct': 78,
 's1_G1_rf_correct': 77,
 's1_G1_lda_correct': 76

In [36]:
with open('precision_offline_sim.txt', 'w') as file:
     file.write(json.dumps(precision_dict))

In [37]:
for subject_number in range(5):
    for gesture_number in range(1,6):
        precision_tree = precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'tree_correct']/precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'total']
        precision_bag  = precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'bag_correct']/precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'total']
        precision_rf   = precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'rf_correct']/precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'total']
        precision_lda  = precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'lda_correct']/precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'total']
        precision_svm  = precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'svm_correct']/precision_dict['s' + str(subject_number) + '_G' + str(gesture_number) + '_' + 'total']
        print('########################')
        print('s' + str(subject_number) + '_G' + str(gesture_number))
        print('########################')
        print('Tree: '+ str(precision_tree))
        print('Bag:  '+ str(precision_bag))
        print('RF:   '+ str(precision_rf))
        print('LDA:  '+ str(precision_lda))
        print('SVM:  '+ str(precision_svm))

########################
s0_G1
########################
Tree: 0.9402985074626866
Bag:  0.8955223880597015
RF:   0.9402985074626866
LDA:  0.9402985074626866
SVM:  0.9402985074626866
########################
s0_G2
########################
Tree: 0.8181818181818182
Bag:  0.09090909090909091
RF:   1.0
LDA:  1.0
SVM:  1.0
########################
s0_G3
########################
Tree: 1.0
Bag:  0.92
RF:   1.0
LDA:  0.98
SVM:  1.0
########################
s0_G4
########################
Tree: 1.0
Bag:  1.0
RF:   1.0
LDA:  1.0
SVM:  1.0
########################
s0_G5
########################
Tree: 0.9871794871794872
Bag:  0.9871794871794872
RF:   1.0
LDA:  1.0
SVM:  1.0
########################
s1_G1
########################
Tree: 0.872093023255814
Bag:  0.9069767441860465
RF:   0.8953488372093024
LDA:  0.8837209302325582
SVM:  0.9767441860465116
########################
s1_G2
########################
Tree: 0.9577464788732394
Bag:  0.7887323943661971
RF:   0.9859154929577465
LDA:  0.8591549295774